In [ ]:
from itertools import combinations
from collections import defaultdict

# -----------------------------
# Input Transactions
# -----------------------------
n = int(input("Enter number of transactions: "))
transactions = []
for i in range(n):
    items = input(f"Enter items for Transaction {i+1} (comma separated): ")
    transactions.append(set(item.strip() for item in items.split(",")))

min_support_count = int(input("Enter Minimum Support (count): "))
total_transactions = len(transactions)

# -----------------------------
# Support Function
# -----------------------------
def get_support(itemset):
    count = 0
    for transaction in transactions:
        if itemset.issubset(transaction):
            count += 1
    support = count / total_transactions
    return count, support

# -----------------------------
# APRIORI ALGORITHM
# -----------------------------
items = set()
for transaction in transactions:
    items.update(transaction)

current_L = []
k = 1
frequent_itemsets = {}

# Generate L1
print("\nL1 Candidates:")
for item in items:
    count, support = get_support({item})
    print(f"{item} -> Count = {count}, Support = {support:.2f}")
    if count >= min_support_count:
        current_L.append(frozenset([item]))

# Generate L2, L3...
while current_L:
    print(f"\nFrequent L{k}:")
    next_L = []

    for itemset in current_L:
        count, support = get_support(set(itemset))
        print(f"{set(itemset)} -> Count = {count}, Support = {support:.2f}")
        frequent_itemsets[itemset] = (count, support)

    k += 1
    candidates = []

    for i in range(len(current_L)):
        for j in range(i+1, len(current_L)):
            union_set = current_L[i].union(current_L[j])
            if len(union_set) == k:
                candidates.append(union_set)

    candidates = list(set(candidates))
    current_L = []

    print(f"\nL{k} Candidates:")
    for candidate in candidates:
        count, support = get_support(set(candidate))
        print(f"{set(candidate)} -> Count = {count}, Support = {support:.2f}")
        if count >= min_support_count:
            current_L.append(candidate)

# -----------------------------
# Association Rules
# -----------------------------
print("\nAssociation Rules:")
for itemset in frequent_itemsets:
    if len(itemset) > 1:
        itemset_count, itemset_support = frequent_itemsets[itemset]
        for i in range(1, len(itemset)):
            for subset in combinations(itemset, i):
                subset = frozenset(subset)
                subset_count, subset_support = get_support(set(subset))
                confidence = itemset_support / subset_support
                print(f"{set(subset)} -> {set(itemset - subset)} | "
                      f"Support = {itemset_support:.2f} | "
                      f"Confidence = {confidence:.2f}")


# -----------------------------
# FP-Tree Node Class
# -----------------------------
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}

# -----------------------------
# Print FP-Tree Properly
# -----------------------------
def print_tree(node, level=0):
    for child in node.children.values():
        print("   " * level + f"|-- {child.item} ({child.count})")
        print_tree(child, level + 1)

# -----------------------------
# Build FP-Tree with Growth Display
# -----------------------------
def build_fp_tree(transactions, min_support):
    freq = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            freq[item] += 1

    freq = {k: v for k, v in freq.items() if v >= min_support}

    print("\nFrequent Items (after min support filtering):")
    for k, v in freq.items():
        print(k, ":", v)

    sorted_items = sorted(freq.items(), key=lambda x: (-x[1], x[0]))
    order = [item[0] for item in sorted_items]

    root = FPNode("NULL", 0, None)
    print("\n---- FP TREE CONSTRUCTION ----")

    for i, transaction in enumerate(transactions):
        filtered = [item for item in transaction if item in freq]
        filtered.sort(key=lambda x: order.index(x))

        current = root
        for item in filtered:
            if item in current.children:
                current.children[item].count += 1
            else:
                current.children[item] = FPNode(item, 1, current)
            current = current.children[item]

        print(f"\nAfter inserting Transaction T{i+1}: {filtered}")
        print_tree(root)

    return root, freq

# -----------------------------
# Generate Frequent Itemsets
# -----------------------------
def generate_frequent_itemsets(transactions, min_support):
    itemsets = defaultdict(int)
    for transaction in transactions:
        for r in range(1, len(transaction)+1):
            for combo in combinations(transaction, r):
                itemsets[frozenset(combo)] += 1

    frequent = {k: v for k, v in itemsets.items() if v >= min_support}
    return frequent

# -----------------------------
# Generate Association Rules (FP)
# -----------------------------
def generate_rules(frequent):
    print("\n---- Association Rules ----")
    for itemset in frequent:
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                for subset in combinations(itemset, i):
                    subset = frozenset(subset)
                    remain = itemset - subset
                    confidence = frequent[itemset] / frequent[subset]
                    print(f"{set(subset)} -> {set(remain)} | "
                          f"Support = {frequent[itemset]} | "
                          f"Confidence = {confidence:.2f}")

# -----------------------------
# MAIN PROGRAM (FP-Growth)
# -----------------------------
transactions_fp = [list(t) for t in transactions]

root, freq = build_fp_tree(transactions_fp, min_support_count)

frequent = generate_frequent_itemsets(transactions_fp, min_support_count)

print("\n---- Frequent Itemsets ----")
for itemset, support in frequent.items():
    print(f"{set(itemset)} : Support = {support}")

generate_rules(frequent)

Enter number of transactions: 6
Enter items for Transaction 1 (comma separated): Milk, Bread, Diaper
Enter items for Transaction 2 (comma separated): Milk, Bread
Enter items for Transaction 3 (comma separated): Bread, Diaper, Beer
Enter items for Transaction 4 (comma separated):  Milk, Diaper, Beer
Enter items for Transaction 5 (comma separated): Milk, Bread, Beer
Enter items for Transaction 6 (comma separated):  Milk, Bread, Diaper, Beer
Enter Minimum Support (count): 2

L1 Candidates:
Beer -> Count = 4, Support = 0.67
Diaper -> Count = 4, Support = 0.67
Bread -> Count = 5, Support = 0.83
Milk -> Count = 5, Support = 0.83

Frequent L1:
{'Beer'} -> Count = 4, Support = 0.67
{'Diaper'} -> Count = 4, Support = 0.67
{'Bread'} -> Count = 5, Support = 0.83
{'Milk'} -> Count = 5, Support = 0.83

L2 Candidates:
{'Diaper', 'Bread'} -> Count = 3, Support = 0.50
{'Beer', 'Milk'} -> Count = 3, Support = 0.50
{'Beer', 'Diaper'} -> Count = 3, Support = 0.50
{'Beer', 'Bread'} -> Count = 3, Support =